# imports

In [1]:
from os import listdir,mkdir,makedirs
from os.path import isfile, join, isdir,exists
import pandas as pd
import numpy as np
from scipy import stats
import re
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from myplots import roundup, rounddown, find_decimal_fold, percentile_cut_off, rarefaction_calc, rarefaction_plot,draw_correlation_scatter
from matplotlib.ticker import FormatStrFormatter
import cPickle as pickle
from Bio.SeqUtils import GC
import seaborn as sns
import random
from scipy.stats import pearsonr
from skbio.diversity.alpha import shannon, simpson, berger_parker_d

from pop_organize import get_sample_data, get_sample_with_dfs
from SufficientStatistics import *
from MyFunctionsShani import *
import math
from myplots import roundup, rounddown, find_decimal_fold
from skbio.stats.distance import mantel
from scipy.spatial.distance import braycurtis, pdist
from GeneralFeaturePhenotypeInteractions.Feature_phenotype_functions import * 
from TCR_microbiome_interactions.TCR_microbiome_interactions_functions import *
from SampleLists.SampleFileFunctions import *

MyPath='/net/mraid08/export/genie/Lab/Personal/ShaniBAF'

done1
stop
stop
done1
stop
done
done


In [2]:
import time
cdate=str(time.strftime("%d%m%Y"))
cdate

'29042018'

# generate microbiome data:


In [ ]:
level='s'
dataFolder='AllSeqProjects'


MPAlevel,MPAlevelBinary=gen_mb_df(level,dataFolder)

In [ ]:
MPAlevel.head()

# filter microbiome data:

## generate updated bd_fd converter based on 'AllSeqProjects' folder

In [ ]:
mergeOn='Year'
readCountFolder='AllSeqProjects'

final_BD_FD_converter=gen_BD_FD_conversion_file(mergeOn,readCountFolder)

## filter Mb data:

In [ ]:
#loading Mb file (RA, based on AllSeqProjects):

file1='%s/MicrobiomeDataTables/MPA_s_AllSeqProjects_RA' %MyPath
MPA_s_AllSeqProjects_RA=pd.read_pickle(file1)

In [ ]:
#load PNP515 sample list:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP515','rb') as fp:
    PNP515=pickle.load(fp)
print len(PNP515)
PNP515[:5]

### filter PNP515 samples, no additional filtering:

In [ ]:
dataDF=MPA_s_AllSeqProjects_RA.reset_index()
dataDFname='MPA_s_AllSeqProjects'
dataMergeOn='FD'
BDFDMergeOn='FD'
SampleList=PNP515
SampleListName='PNP515'
filterGenotek=False
filterMinimalReads=None
folderToSave='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/MicrobiomeDataTables/Filtered'



filter_data_on_FDproperties(dataDF,dataDFname,dataMergeOn,BDFDMergeOn,SampleList,SampleListName,
                            filterGenotek, filterMinimalReads, folderToSave,BD_FD=None)

### filter PNP515 samples, only swabs, minimal reads=9000000

In [ ]:
dataDF=MPA_s_AllSeqProjects_RA.reset_index()
dataDFname='MPA_s_AllSeqProjects'
dataMergeOn='FD'
BDFDMergeOn='FD'
SampleList=PNP515
SampleListName='PNP515'
filterGenotek=True
filterMinimalReads=9000000
folderToSave='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/MicrobiomeDataTables/Filtered'



filter_data_on_FDproperties(dataDF,dataDFname,dataMergeOn,BDFDMergeOn,SampleList,SampleListName,
                            filterGenotek, filterMinimalReads, folderToSave,BD_FD=None)

# merge on BDs:

no need to filter outliers and same person, as this is done in subsequent functions

In [ ]:
#non filtered file:

f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/MicrobiomeDataTables/Filtered/MPA_s_AllSeqProjects_SampleListPNP515_filterGenotekFalse_filterMinimalReadNone'
MPA515notfiltered=pd.read_pickle(f1)


DFtoGroup=MPA515notfiltered
DFtoGroupName='MPA515notfiltered'
groupFunction='mean'
filterOutlierSamples=False
filterSamePerson=False
folderToSave='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/MicrobiomeDataTables/FilteredAndMergedOnBD'
nSTD=None,
nMinSamples=None


groupedDF=mergeOnBD(DFtoGroup,DFtoGroupName,groupFunction,filterOutlierSamples,filterSamePerson,folderToSave,nSTD=None,
              nMinSamples=None)

In [ ]:
#swabs only,min9k reads filtered file:

f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/MicrobiomeDataTables/Filtered/MPA_s_AllSeqProjects_SampleListPNP515_filterGenotekTrue_filterMinimalRead9000000'
MPA515swabsMin9k=pd.read_pickle(f1)


DFtoGroup=MPA515swabsMin9k
DFtoGroupName='MPA515swabsMin9k'
groupFunction='mean'
filterOutlierSamples=False
filterSamePerson=False
folderToSave='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/MicrobiomeDataTables/FilteredAndMergedOnBD'
nSTD=None,
nMinSamples=None


groupedDF=mergeOnBD(DFtoGroup,DFtoGroupName,groupFunction,filterOutlierSamples,filterSamePerson,folderToSave,nSTD=None,
              nMinSamples=None)

# TCR-mb interaction tests:

## load and prepare files for analysis:

### load and edit Mb files

In [ ]:
#load BD_FD converter, to make sure we remove BD_FD columns from the MB file:
f1='%s/Sample files/updatedBDandFDlists/final_BD_FD_converter_mergedOnYear_RCfolderAllSeqProjects_17042018' %MyPath
BD_FD=pd.read_pickle(f1)

In [ ]:
def edit_MbDF_temp(MbDF):
    try:
        MbDF=MbDF.set_index('BD') #### very important!!
    except KeyError:
        print 'couldnt reset index to BD,probably BD is already the index'

    #remove unnecessary columns:
    for column in BD_FD.columns.values:
        if column in MbDF.columns.values:
            print 'dropping %s column' %column
            MbDF=MbDF.drop(column,axis=1)
            
    return MbDF

In [ ]:
# load non-filtered file and edit:
f1='%s/MicrobiomeDataTables/FilteredAndMergedOnBD/MPA515notfiltered_meannSTDNonenMinSamplesNone_filterOutlierSampleFalse_filterSamePersonFalse' %MyPath
Mb_PNP515_notFiltered=pd.read_pickle(f1)

Mb_PNP515_notFiltered=edit_MbDF_temp(Mb_PNP515_notFiltered)

In [ ]:
# load filtered file and edit:
f1='%s/MicrobiomeDataTables/FilteredAndMergedOnBD/MPA515swabsMin9k_meannSTDNonenMinSamplesNone_filterOutlierSampleFalse_filterSamePersonFalse' %MyPath
Mb_PNP515_swabsMin9k=pd.read_pickle(f1)

Mb_PNP515_swabsMin9k=edit_MbDF_temp(Mb_PNP515_swabsMin9k)

### load and edit TCR file:

In [ ]:
#load PNP515, 
file1 = '%s/TCR_real_data/sharingAnalysis/sharingMatrix_PNP515_minNshared50_RA' %MyPath
sharingMatrix_PNP515_minNshared50_RA = pd.read_pickle(file1)
sharingMatrix_PNP515_minNshared50_RA.head()

## run RA correlation test:

### run trial:

In [ ]:
datasetFolder='%s/TCR_real_data' %MyPath


MbDF_RA=Mb_PNP515_swabsMin9k
MbDFName='MbPNP515swabsMin9k'
TCRdf_RA=sharingMatrix_PNP515_minNshared50_RA
TCRdfName='sharingMatrixPNP515minNshared50'
NsharedSamplesForSpecies=None
NsharedSamplesForSeqs=None
topNspecies=10
topNseqs=10
outlierSTD=3
completePearson=True
pearsonForHeatMap=False





PearsonResults_full=find_associations_TCR_RA_MBspecies_RA(MbDF_RA, MbDFName,TCRdf_RA, TCRdfName, datasetFolder,NsharedSamplesForSpecies,NsharedSamplesForSeqs,
                                          topNspecies, topNseqs,outlierSTD,completePearson,pearsonForHeatMap)

### run for top 250 seqs and species:

In [ ]:
datasetFolder='%s/TCR_real_data' %MyPath


MbDF_RA=Mb_PNP515_swabsMin9k
MbDFName='MbPNP515swabsMin9k'
TCRdf_RA=sharingMatrix_PNP515_minNshared50_RA
TCRdfName='sharingMatrixPNP515minNshared50'
NsharedSamplesForSpecies=None
NsharedSamplesForSeqs=None
topNspecies=250
topNseqs=250
outlierSTD=3
completePearson=True
pearsonForHeatMap=False





PearsonResults_full=find_associations_TCR_RA_MBspecies_RA(MbDF_RA, MbDFName,TCRdf_RA, TCRdfName, datasetFolder,NsharedSamplesForSpecies,NsharedSamplesForSeqs,
                                          topNspecies, topNseqs,outlierSTD,completePearson,pearsonForHeatMap)

In [ ]:
datasetFolder='%s/TCR_real_data' %MyPath


MbDF_RA=Mb_PNP515_swabsMin9k
MbDFName='MbPNP515swabsMin9k'
TCRdf_RA=sharingMatrix_PNP515_minNshared50_RA
TCRdfName='sharingMatrixPNP515minNshared50'
NsharedSamplesForSpecies=None
NsharedSamplesForSeqs=None
topNspecies=400
topNseqs=250
outlierSTD=3
completePearson=True
pearsonForHeatMap=False





PearsonResults_full=find_associations_TCR_RA_MBspecies_RA(MbDF_RA, MbDFName,TCRdf_RA, TCRdfName, datasetFolder,NsharedSamplesForSpecies,NsharedSamplesForSeqs,
                                          topNspecies, topNseqs,outlierSTD,completePearson,pearsonForHeatMap)

## run binary fisher test:

### run trial:

In [ ]:
datasetFolder='%s/TCR_real_data' %MyPath

MbDF_RA=Mb_PNP515_swabsMin9k
MbDFName='MbPNP515swabsMin9k'
TCRdf_RA=sharingMatrix_PNP515_minNshared50_RA
TCRdfName='sharingMatrixPNP515minNshared50'
NsharedSamplesForSpecies=None
NsharedSamplesForSeqs=None
topNspecies=50
topNseqs=50



find_associations_binaryTCR_binaryMBspecies(MbDF_RA, MbDFName,TCRdf_RA, TCRdfName, datasetFolder,NsharedSamplesForSpecies,NsharedSamplesForSeqs,
                                          topNspecies, topNseqs)

### runfor top 250 seq and species

In [ ]:
datasetFolder='%s/TCR_real_data' %MyPath

MbDF_RA=Mb_PNP515_swabsMin9k
MbDFName='MbPNP515swabsMin9k'
TCRdf_RA=sharingMatrix_PNP515_minNshared50_RA
TCRdfName='sharingMatrixPNP515minNshared50'
NsharedSamplesForSpecies=None
NsharedSamplesForSeqs=None
topNspecies=250
topNseqs=250



find_associations_binaryTCR_binaryMBspecies(MbDF_RA, MbDFName,TCRdf_RA, TCRdfName, datasetFolder,NsharedSamplesForSpecies,NsharedSamplesForSeqs,
                                          topNspecies, topNseqs)

## merge pearson and fisher results:

In [ ]:
#load pearson results:

datasetFolder='%s/TCR_real_data' %MyPath
MbDFName='MbPNP515swabsMin9k'
TCRdfName='sharingMatrixPNP515minNshared50'
NsharedSamplesForSpecies=None
NsharedSamplesForSeqs=None
topNspecies=250
topNseqs=250
outlierSTD=3


resultName='%s_%s_NsharedSpecies%s_Nsharedseqs%s_topSpecies%s_topSeqs%s_outlierSTD%s' \
%(MbDFName,TCRdfName,NsharedSamplesForSpecies,NsharedSamplesForSeqs,topNspecies,topNseqs,outlierSTD)

resultFolder='%s/TCR_mb_results/PearsonResults_%s' %(datasetFolder,resultName)
resultFile='%s/PearsonResults.xlsx' %resultFolder
Pearson_results=pd.read_excel(resultFile)
print len(Pearson_results)
Pearson_results.head()

In [ ]:
#load Fisher results:

datasetFolder='%s/TCR_real_data' %MyPath
MbDFName='MbPNP515swabsMin9k'
TCRdfName='sharingMatrixPNP515minNshared50'
NsharedSamplesForSpecies=None
NsharedSamplesForSeqs=None
topNspecies=250
topNseqs=250
outlierSTD=3


resultName='%s_%s_NsharedSpecies%s_Nsharedseqs%s_topSpecies%s_topSeqs%s' \
%(MbDFName,TCRdfName,NsharedSamplesForSpecies,NsharedSamplesForSeqs,topNspecies,topNseqs)

resultFolder='%s/TCR_mb_results/FisherResults_%s' %(datasetFolder,resultName)
resultFile='%s/FisherResults.xlsx' %resultFolder
Fisher_results=pd.read_excel(resultFile)
print len(Fisher_results)
Fisher_results.head()

In [ ]:
#merge:
merged_fisher_pearson=pd.merge(Pearson_results,
                               Fisher_results,how='inner',
                              left_on=('species','seq'),right_on=('species','seq'))
print len(merged_fisher_pearson)
for column in merged_fisher_pearson:
    newColumn=column.replace('_x','_pearson')
    newColumn=newColumn.replace('_y','_Fisher')
    merged_fisher_pearson=merged_fisher_pearson.rename(columns={column:newColumn})
merged_fisher_pearson=merged_fisher_pearson.sort_values(by='p_pearson')
merged_fisher_pearson.head()

## add identity information to the table:

In [ ]:
#load TCR identity table:
file1= '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR CDR3 sequence databases/CDR3identityTable_22022018'
CDR3identityTable=pd.read_pickle(file1)
CDR3identityTable.head()

In [ ]:
merged_fisher_pearson_withIdentity=pd.merge(merged_fisher_pearson,CDR3identityTable,how='left',
                                           left_on='seq',right_index=True)
print len(merged_fisher_pearson)
print len(CDR3identityTable)
print len(merged_fisher_pearson_withIdentity)
merged_fisher_pearson_withIdentity.head()

In [ ]:
merged_fisher_pearson_withIdentity[merged_fisher_pearson_withIdentity['Epitope species_VDJDB'].notnull()].head()

In [ ]:


file1='%s/TCR_mb_results/mergedFisherPearsonWithIdentity_%s' %(datasetFolder,resultName)
merged_fisher_pearson_withIdentity.to_pickle(file1)

file2='%s/TCR_mb_results/mergedFisherPearsonWithIdentity_%s.xlsx' %(datasetFolder,resultName)
merged_fisher_pearson_withIdentity.to_excel(file2)



# mega function:

the function TCR_Mb_interactions_for_dataset was copied to TCR_microbiome_interactions_functions.py

### run for PNP515

In [ ]:
#load PNP515 sample list:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP515','rb') as fp:
    PNP515=pickle.load(fp)
print len(PNP515)
print PNP515[:5]


datasetFolder='%s/TCR_real_data' %MyPath
datasetName='PNP515'

gen_mb=False
mbLevel='s'
mbDataFolder='AllSeqProjects'
SampleList=PNP515
SampleListName='PNP515'
filterGenotek=True
filterMinimalReads=9000000

# (NsharedSamplesForSpecies,NsharedSamplesForSeqs,topNspecies,topNseqs):
thresholdList=[(None,None,250,250)]
outlierSTD=3
completePearson=True
pearsonForHeatMap=False
corrTest='pearson'

merged_fisher_pearson_withIdentity=TCR_Mb_interactions_for_dataset(datasetFolder,datasetName,gen_mb,mbLevel,mbDataFolder,SampleList,
                                    SampleListName, filterGenotek,filterMinimalReads,
                                    thresholdList,outlierSTD,corrTest,completePearson,
                                    pearsonForHeatMap)




##### run for PNP515_ss15000_rep2 ***with spearman correlation test***

In [ ]:
#load PNP515 sample list:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP515','rb') as fp:
    PNP515=pickle.load(fp)
print len(PNP515)
print PNP515[:5]


datasetFolder='%s/TCR_real_data/SubSampled15000data_rep2' %MyPath
datasetName='PNP515_ss15000_rep2'

gen_mb=False
mbLevel='s'
mbDataFolder='AllSeqProjects'
SampleList=PNP515
SampleListName='PNP515'
filterGenotek=True
filterMinimalReads=9000000

# (NsharedSamplesForSpecies,NsharedSamplesForSeqs,topNspecies,topNseqs):
thresholdList=[(None,None,250,250)]
outlierSTD=3
completePearson=True
pearsonForHeatMap=False
corrTest='spearman'

merged_fisher_pearson_withIdentity=TCR_Mb_interactions_for_dataset(datasetFolder,datasetName,gen_mb,mbLevel,mbDataFolder,SampleList,
                                    SampleListName, filterGenotek,filterMinimalReads,
                                    thresholdList,outlierSTD,corrTest,completePearson,
                                    pearsonForHeatMap)


### run for PNP515_ss18000_rep2 ***with pearson correlation test***

In [ ]:
#load PNP515 sample list:
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP515','rb') as fp:
    PNP515=pickle.load(fp)
print len(PNP515)
print PNP515[:5]


datasetFolder='%s/TCR_real_data/SubSampled18000data_rep2' %MyPath
datasetName='PNP515_ss18000_rep2'

gen_mb=False
mbLevel='s'
mbDataFolder='AllSeqProjects'
SampleList=PNP515
SampleListName='PNP515'
filterGenotek=True
filterMinimalReads=9000000

# (NsharedSamplesForSpecies,NsharedSamplesForSeqs,topNspecies,topNseqs):
thresholdList=[(None,None,50,50)]
outlierSTD=3
completePearson=True
pearsonForHeatMap=False
corrTest='pearson'

merged_fisher_pearson_withIdentity=TCR_Mb_interactions_for_dataset(datasetFolder,datasetName,gen_mb,mbLevel,mbDataFolder,SampleList,
                                    SampleListName, filterGenotek,filterMinimalReads,
                                    thresholdList,outlierSTD,corrTest,completePearson,
                                    pearsonForHeatMap)


# develop permutation corr analysis

In [ ]:
#load mb file:
print 'loading most updated Mb file...'
f1 = '%s/MicrobiomeDataTables/FilteredAndMergedOnBD/MPA_s_AllSeqProjects_SampleListPNP515_filterGenotekTrue_filterMinimalRead9000000_meannSTDNonenMinSamplesNone_filterOutlierSampleFalse_filterSamePersonFalse' % MyPath
groupedFilteredMB = pd.read_pickle(f1)
filterGenotek = True
filterMinimalReads = 9000000

In [ ]:
# (2) prepare MB and TCR files for analysis:
print 'preparing mb and TCR files for analysis:...'

# prepare MB file:
# load BD_FD converter, to make sure we remove BD_FD columns from the MB file:
f1 = '%s/Sample files/updatedBDandFDlists/final_BD_FD_converter_mergedOnYear_RCfolderAllSeqProjects_17042018' % MyPath
BD_FD = pd.read_pickle(f1)

try:
    MbDF_RA = groupedFilteredMB.set_index('BD')  #### very important!!
except KeyError:
    print 'couldnt reset index to BD,probably BD is already the index'
    MbDF_RA = groupedFilteredMB


# remove unnecessary columns:
for column in BD_FD.columns.values:
    if column in MbDF_RA.columns.values:
        print 'dropping %s column' % column
        MbDF_RA = MbDF_RA.drop(column, axis=1)

# prepare TCR file:
minNshared = 50  # add to function variables if necessary
file1 = '%s/sharingAnalysis/sharingMatrix_%s_minNshared%s_RA' % (datasetFolder, datasetName, minNshared)
TCRdf_RA = pd.read_pickle(file1)

In [ ]:
# (3) run tests and merge results:
print 'running interaction tests...'

MbDFName = 'Mb_Swabs%sMinReads%s' % (filterGenotek, filterMinimalReads)
TCRdfName = 'TCRminNshared%s' % minNshared

# load TCR identity table:
file1 = '/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR CDR3 sequence databases/CDR3identityTable_22022018'
CDR3identityTable = pd.read_pickle(file1)
CDR3identityTable.head()

In [ ]:
NsharedSamplesForSpecies = None
NsharedSamplesForSeqs =None
topNspecies = 50
topNseqs = 50



In [ ]:
#get seq-species df:
# get combined df of seqs and species according to the requested thresholds:
print 'preparing combined seq and species file...'
species_sequences_df, speciesList, sequenceList = prepare_MbDF_and_TCRdf_for_analysis(MbDF_RA, MbDFName, TCRdf_RA, TCRdfName, NsharedSamplesForSpecies, NsharedSamplesForSeqs,
                                      topNspecies, topNseqs)

In [ ]:
### shuffle this df, and run correlation for heatmap on each permutation:



binary analysis-fisher : correct function in py file (use the new sub-function)
merging data
add TCR identity
look at results
mega function: (specific dataset, generate sharing matrix-yes/no (if not,use full dataset, min50), generate Mb table-yes/no(if not, load 's',RA,AllSeqProjects),define filtering, define thresholds for analysis)

In [6]:
file1='%s/TCR_real_data/SubSampled15000data_rep2/TCR_mb_results/\
permCorrTest_NoneNone250250_stdNone/Final_permCorrResults_9998Perms_df.xlsx' %MyPath
df=pd.read_excel(file1)
df.head()

,species,seq,r_real,nSamplesSeq,nSamplesSpecies,nComSeqSpecies,real_p_Corr9998Perms,Sig by bonferroni corrected pVal,sig. by FDR=0.01,sig. by FDR=0.1,Unnamed: 10,Unnamed: 11,Unnamed: 12
12998,Prevotella_copri,CASSLSGYEQYF,-0.208157,50,220,24,0.0002,0,0,0,NaN,NaN,NaN
1007,Eubacterium_hallii,CASSDRDTGELFF,-0.206572,132,333,132,0.0002,0,0,0,NaN,NaN,NaN
11943,Akkermansia_muciniphila,CASSLGTGSYEQYF,-0.197068,57,227,28,0.0003,0,0,0,NaN,NaN,NaN
18505,Lachnospiraceae_bacterium_8_1_57FAA,CASSLGGNTEAFF,-0.196836,136,154,50,0.0005,0,0,0,NaN,NaN,NaN
16283,Escherichia_unclassified,CASSLGGTEAFF,-0.194766,91,176,33,0.0004,0,0,0,NaN,NaN,NaN


In [10]:
df=add_corrected_pValues(df,
        pValueColumn='real_p_Corr9998Perms',nTests=62500,FDR=0.25)
df.head()

,species,seq,r_real,nSamplesSeq,nSamplesSpecies,nComSeqSpecies,real_p_Corr9998Perms,Sig by bonferroni corrected pVal,sig. by FDR=0.01,sig. by FDR=0.1,Unnamed: 10,Unnamed: 11,Unnamed: 12,sig. by FDR=0.25
12998,Prevotella_copri,CASSLSGYEQYF,-0.208157,50,220,24,0.0002,0,0,0,NaN,NaN,NaN,0
1007,Eubacterium_hallii,CASSDRDTGELFF,-0.206572,132,333,132,0.0002,0,0,0,NaN,NaN,NaN,0
11943,Akkermansia_muciniphila,CASSLGTGSYEQYF,-0.197068,57,227,28,0.0003,0,0,0,NaN,NaN,NaN,0
18505,Lachnospiraceae_bacterium_8_1_57FAA,CASSLGGNTEAFF,-0.196836,136,154,50,0.0005,0,0,0,NaN,NaN,NaN,0
16283,Escherichia_unclassified,CASSLGGTEAFF,-0.194766,91,176,33,0.0004,0,0,0,NaN,NaN,NaN,0


In [13]:
df.sort_values(by='real_p_Corr9998Perms').head(10)

,species,seq,r_real,nSamplesSeq,nSamplesSpecies,nComSeqSpecies,real_p_Corr9998Perms,Sig by bonferroni corrected pVal,sig. by FDR=0.01,sig. by FDR=0.1,Unnamed: 10,Unnamed: 11,Unnamed: 12,sig. by FDR=0.25
7084,Alistipes_putredinis,CASSLKETQYF,0.200318,80,291,75,0.0001,0,0,0,NaN,NaN,0.0001,0
12998,Prevotella_copri,CASSLSGYEQYF,-0.208157,50,220,24,0.0002,0,0,0,NaN,NaN,NaN,0
58108,Peptostreptococcus_anaerobius,CASSPGQGYGYTF,0.232935,67,8,6,0.0002,0,0,0,NaN,NaN,NaN,0
61559,Eubacterium_sp_3_1_31,CASSFGETQYF,0.229159,82,7,6,0.0002,0,0,0,NaN,NaN,NaN,0
49616,Eubacterium_cylindroides,CASSLDRNYGYTF,0.224770,69,15,9,0.0002,0,0,0,NaN,NaN,NaN,0
52612,Lachnospiraceae_bacterium_9_1_43BFAA,CASSLGQGTDTQYF,0.222659,70,17,10,0.0002,0,0,0,NaN,NaN,NaN,0
2624,Bacteroides_vulgatus,CASSEASGGADTQYF,0.201354,70,324,70,0.0002,0,0,0,NaN,NaN,NaN,0
25176,Bacteroides_intestinalis,CASSLGAYEQYF,0.199070,60,96,29,0.0002,0,0,0,NaN,NaN,NaN,0
56957,Porphyromonas_bennonis,CASSLGATNEKLFF,0.240927,57,10,7,0.0002,0,0,0,NaN,NaN,NaN,0
61463,Collinsella_tanakaei,CASSLGQGNEKLFF,0.257031,52,8,6,0.0002,0,0,0,NaN,NaN,NaN,0


In [14]:
df=df.drop(['Unnamed: 10','Unnamed: 11','Unnamed: 12'],axis=1)

In [15]:
df.to_excel(file1)

In [16]:
df.head()

,species,seq,r_real,nSamplesSeq,nSamplesSpecies,nComSeqSpecies,real_p_Corr9998Perms,Sig by bonferroni corrected pVal,sig. by FDR=0.01,sig. by FDR=0.1,sig. by FDR=0.25
12998,Prevotella_copri,CASSLSGYEQYF,-0.208157,50,220,24,0.0002,0,0,0,0
1007,Eubacterium_hallii,CASSDRDTGELFF,-0.206572,132,333,132,0.0002,0,0,0,0
11943,Akkermansia_muciniphila,CASSLGTGSYEQYF,-0.197068,57,227,28,0.0003,0,0,0,0
18505,Lachnospiraceae_bacterium_8_1_57FAA,CASSLGGNTEAFF,-0.196836,136,154,50,0.0005,0,0,0,0
16283,Escherichia_unclassified,CASSLGGTEAFF,-0.194766,91,176,33,0.0004,0,0,0,0
